In [3]:
# import networkx as nx
import pickle
from pathlib import Path
from scipy.spatial import distance

In [16]:
camId = 'c042'
fp = Path('../../datasets/detect_merge/{}/{}_mot_feat.pkl'.format(camId,camId))
cam = pickle.load(fp.open('rb'))
cam

{'c042_1_0.png': {'bbox': (912.0, 400.0, 1044.0, 512.0),
  'frame': 'img000000',
  'id': 1,
  'imgname': 'c042_1_0.png',
  'feat': array([-0.02163875, -0.06552612, -0.00827877, ...,  0.00435317,
         -0.04050457, -0.01504902])},
 'c042_1_1.png': {'bbox': (912.0, 400.0, 1044.0, 512.0),
  'frame': 'img000001',
  'id': 1,
  'imgname': 'c042_1_1.png',
  'feat': array([-0.02170294, -0.06552588, -0.00828269, ...,  0.00435916,
         -0.04047706, -0.01504402])},
 'c042_1_10.png': {'bbox': (911.0, 400.0, 1045.0, 512.0),
  'frame': 'img000010',
  'id': 1,
  'imgname': 'c042_1_10.png',
  'feat': array([-0.02193541, -0.06584657, -0.00868035, ...,  0.00446968,
         -0.04020796, -0.01520642])},
 'c042_1_100.png': {'bbox': (911.0, 400.0, 1045.0, 512.0),
  'frame': 'img000100',
  'id': 1,
  'imgname': 'c042_1_100.png',
  'feat': array([-0.01916933, -0.06661613, -0.00717728, ...,  0.00576698,
         -0.0421818 , -0.01736925])},
 'c042_1_101.png': {'bbox': (911.0, 400.0, 1045.0, 512.0),
  '

In [15]:
ls = []
for value in cam.values():
    ls.append(value['id'])
max(ls)

1744

In [17]:
from collections import Counter
Counterter(ls)

Counter({1: 510,
         2: 42,
         3: 41,
         4: 961,
         5: 520,
         8: 95,
         17: 8,
         23: 2,
         27: 4,
         28: 25,
         29: 1,
         32: 18,
         33: 15,
         36: 1,
         38: 1,
         40: 25,
         42: 69,
         44: 1,
         45: 822,
         48: 12,
         49: 1,
         50: 4,
         52: 7,
         55: 1410,
         59: 1,
         60: 60,
         62: 27,
         64: 1,
         72: 2,
         73: 35,
         81: 24,
         86: 1,
         90: 753,
         93: 1,
         103: 1,
         104: 8,
         105: 35,
         110: 31,
         112: 26,
         125: 64,
         137: 1,
         138: 1,
         140: 89,
         145: 32,
         147: 3,
         148: 37,
         152: 33,
         153: 20,
         162: 1,
         166: 4,
         167: 4,
         169: 4,
         170: 5,
         171: 3,
         173: 2,
         180: 3,
         181: 3,
         183: 3,
         184: 87,
 

In [97]:
### read pickle file
def compute_distance(current_graph, threshold=10):
    # compute distance for all edge in graph
    nodes = list(current_graph.nodes())
    for node1 in nodes:
        for node2 in nodes:
            if node1 != node2:
                dis = distance.euclidean(current_graph.nodes[node1]['cnn_feat'], current_graph.nodes[node2]['cnn_feat'])
                if dis < threshold:
                    current_graph.add_edge(node1, node2, distance = dis)
    return current_graph

def runing_all(camId):
    # Create saving dir:
    p = Path(camId)
    p.mkdir(parents=True, exist_ok=True)
    
    # Read pickle file
    fp = Path('../../datasets/detect_merge/{}/{}_mot_feat.pkl'.format(camId,camId))
    cam = pickle.load(fp.open('rb'))
    bboxes = list(cam.keys())
    # bboxes = bboxes[0:1000]

    # Extract Graph
    previous_trackletId = 0
    ending_bbox = bboxes[-1]
    for bbox in bboxes:
        # print(bbox)
        current_trackletId = bbox.split("_")[1]
        if current_trackletId == previous_trackletId:
            # add node
            current_graph.add_node(bbox, cnn_feat = cam[bbox]['feat'])
        else:
            # compute distance, add edges and save graph
            if previous_trackletId != 0:
                # print("saving")
                current_graph = compute_distance(current_graph)
                nx.write_gpickle(current_graph, '{}/{}_{}.gpickle'.format(camId, camId, previous_trackletId))
            # Create new Graph
            current_graph = nx.Graph()
            # add node to new graph
            current_graph.add_node(bbox, cnn_feat = cam[bbox]['feat'])
            # update trackletID
            previous_trackletId = current_trackletId
        if bbox == ending_bbox:
            # compute distance and add edges
            current_graph = compute_distance(current_graph)
            # Save graph
            nx.write_gpickle(current_graph, '{}/{}_{}.gpickle'.format(camId, camId, previous_trackletId))



In [98]:
camIds = ['c041','c042','c043','c044','c045','c046']
for camId in camIds:
    runing_all(camId)

In [6]:
current_graph = nx.read_gpickle("c041/c041_1012.gpickle")

In [12]:
current_graph.nodes['c041_1012_1230.png']['cnn_feat'].shape

(2048,)

In [7]:
current_graph.nodes(data=True)

NodeDataView({'c041_1012_1230.png': {'cnn_feat': array([-0.0112076 , -0.01595505, -0.02589529, ...,  0.02090746,
       -0.00204681, -0.00604074])}, 'c041_1012_1231.png': {'cnn_feat': array([-0.01083438, -0.01692321, -0.02536282, ...,  0.02100751,
       -0.00380058, -0.00643304])}, 'c041_1012_1239.png': {'cnn_feat': array([-0.00335926, -0.02168168, -0.03037474, ...,  0.02285832,
       -0.0268786 , -0.01118405])}, 'c041_1012_1240.png': {'cnn_feat': array([-0.0042715 , -0.02270086, -0.03071924, ...,  0.02293945,
       -0.02879026, -0.01171308])}, 'c041_1012_1241.png': {'cnn_feat': array([-0.00560035, -0.02269921, -0.03030109, ...,  0.02264109,
       -0.02973167, -0.01146378])}, 'c041_1012_1242.png': {'cnn_feat': array([-0.00566669, -0.02413207, -0.03077428, ...,  0.02259981,
       -0.03149925, -0.01118974])}, 'c041_1012_1243.png': {'cnn_feat': array([-0.00537545, -0.0255689 , -0.03184935, ...,  0.02217709,
       -0.03242671, -0.01113434])}, 'c041_1012_1244.png': {'cnn_feat': array(

In [8]:
current_graph.edges(data=True)

EdgeDataView([('c041_1012_1230.png', 'c041_1012_1231.png', {'distance': 0.07017641668054142}), ('c041_1012_1230.png', 'c041_1012_1239.png', {'distance': 0.38277362446661545}), ('c041_1012_1230.png', 'c041_1012_1240.png', {'distance': 0.41509276335139483}), ('c041_1012_1230.png', 'c041_1012_1241.png', {'distance': 0.44719791963120825}), ('c041_1012_1230.png', 'c041_1012_1242.png', {'distance': 0.4752903585871868}), ('c041_1012_1230.png', 'c041_1012_1243.png', {'distance': 0.5087336092782422}), ('c041_1012_1230.png', 'c041_1012_1244.png', {'distance': 0.5432657571276027}), ('c041_1012_1230.png', 'c041_1012_1245.png', {'distance': 0.574705705248928}), ('c041_1012_1230.png', 'c041_1012_1246.png', {'distance': 0.5959062443817301}), ('c041_1012_1230.png', 'c041_1012_1248.png', {'distance': 0.6190571451740738}), ('c041_1012_1230.png', 'c041_1012_1249.png', {'distance': 0.6334316984554419}), ('c041_1012_1230.png', 'c041_1012_1250.png', {'distance': 0.6528612674131046}), ('c041_1012_1230.png', 